In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.data_sources import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

In [ ]:
candles = await clob.get_candles_last_days("binance", "SOL-USDT", "1m", 10)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from core.data_structures.candles import Candles

def plot_trend_fury(candles: Candles, candles_with_trend: pd.DataFrame):
    # Create the figure with two rows
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
        row_heights=[0.7, 0.3],
        specs=[[{"secondary_y": False}], [{"secondary_y": True}]]
    )

    # Add candlestick chart to the first row
    candlestick = candles.candles_trace()
    fig.add_trace(candlestick, row=1, col=1)

    # Add VWAP to the first row
    fig.add_trace(
        go.Scatter(
            x=candles_with_trend.index,
            y=candles_with_trend['vwap'],
            mode='lines',
            name='VWAP',
            line=dict(color="#FF4500", width=2)
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=candles_with_trend.index,
            y=candles_with_trend['rolling_vwap'],
            mode='lines',
            name='Rolling VWAP',
            line=dict(color="#FF4500", width=2, dash="dash")
        ),
        row=1, col=1
    )
    # Add the signal to the second row (primary y-axis)
    fig.add_trace(
        go.Scatter(
            x=candles_with_trend.index,
            y=candles_with_trend['signal'],
            mode='lines',
            name='Signal',
            line=dict(color="#00FFFF", width=2)
        ),
        row=2, col=1, secondary_y=False
    )

    # Add the slope to the second row (secondary y-axis)
    fig.add_trace(
        go.Scatter(
            x=candles_with_trend.index,
            y=candles_with_trend['slope'],
            mode='lines',
            name='Slope',
            line=dict(color="#FF1493", width=2)
        ),
        row=2, col=1, secondary_y=True
    )

    fig.add_trace(
        go.Scatter(
            x=candles_with_trend.index,
            y=candles_with_trend['cumulative_slope_diff'],
            mode='lines',
            name='Cumulative Slope Difference',
            line=dict(color="#FFD700", width=2, dash="dash")
        ),
        row=2, col=1, secondary_y=True
    )
    # Adding buy signals
    fig.add_trace(go.Scatter(
        x=candles_with_trend[candles_with_trend['signal'] == 1].index,
        y=candles_with_trend[candles_with_trend['signal'] == 1]['close'],
        mode='markers',
        marker=dict(symbol='triangle-up', color='green', size=10),
        name='Buy Signal'
    ))
    
    # Adding sell signals
    fig.add_trace(go.Scatter(
        x=candles_with_trend[candles_with_trend['signal'] == -1].index,
        y=candles_with_trend[candles_with_trend['signal'] == -1]['close'],
        mode='markers',
        marker=dict(symbol='triangle-down', color='red', size=10),
        name='Sell Signal'
    ))

    # Update layout for dark theme and technical colors
    fig.update_layout(
        height=800,
        width=1800,
        title_text="Candlestick Chart with Trend Fury Indicators and VWAP",
        xaxis_rangeslider_visible=False,
        template="plotly_dark",
        hovermode="x unified",
        hoverdistance=100,
        spikedistance=1000,
    )

    # Update y-axis labels and colors
    fig.update_yaxes(title_text="Price", row=1, col=1, color="#FFFFFF")
    fig.update_yaxes(title_text="Signal", row=2, col=1, color="#00FFFF")
    fig.update_yaxes(title_text="Slopes", row=2, col=1, secondary_y=True, color="#FF69B4")

    # Update x-axis color and add spikes
    fig.update_xaxes(
        color="#FFFFFF",
        showspikes=True,
        spikecolor="white",
        spikesnap="cursor",
        spikemode="across"
    )

    return fig


In [ ]:
from core.features.candles.trend_fury import TrendFury, TrendFuryConfig

window = 10
vwap_window = 500
use_returns = False
use_ema = True
use_volume_weighting = True
volume_normalization_window = 50
reversal_sensitivity = 0.05
cum_diff_quantile_threshold = 0.8
slope_quantile_threshold = 0.05
use_vwap_filter = False
use_slope_filter = True


# Create an instance of TrendFury with default configuration
trend_fury = TrendFury(TrendFuryConfig(window=window, vwap_window=vwap_window, use_returns=use_returns,  use_ema=True,
                                       use_volume_weighting=use_volume_weighting,
                                       volume_normalization_window=volume_normalization_window,
                                       cum_diff_quantile_threshold=cum_diff_quantile_threshold,
                                       reversal_sensitivity=reversal_sensitivity,
                                       use_vwap_filter=use_vwap_filter,
                                       use_slope_filter=use_slope_filter,
                                       slope_quantile_threshold=slope_quantile_threshold))

# Apply TrendFury to the candles data
candles_with_trend = candles.add_feature(trend_fury).data



# Calculate the signal

fig = plot_trend_fury(candles, candles_with_trend)

# Show the figure
fig.show()